### MC Simulation BlackJack State-Value
---
- dealer policy: HIT17
- player policy: sticks on 20 or 21, otherwise hit
---
**States:** player's current sum(12–21), the dealer’s one showing card (ace–10), player's usable ace

In [6]:
import numpy as np

In [9]:
class BlackJackMC(object):
    def __init__(self):
        self.state_value = {}
    
    # give card
    @staticmethod
    def giveCard():
        # 1 stands for ace
        c_list = list(range(1, 11)) + [10, 10, 10]
        return np.random.choice(c_list)
    
    def dealerPolicy(self, current_value, has_ace, is_end):
        if current_value > 21:
            if has_ace:
                current_value -= 10
            else:
                return current_value, has_ace, True
                    
        if current_value >= 17:
            return current_value, has_ace, is_end
        else:
            card = giveCard()
            if card == 1:
                if current_value <= 10:
                    return current_value+11, True, False
                return current_value+1, True, False
            else:
                return current_value+card, has_ace, False
            
        
#         while True:
#             current_card = giveCard()
#             if current_card == 1:
#                 has_ace = True
#             value_sum += 
#             if value_sum >= 17:
#                 break
                
    def playerPolicy(self, current_value, has_ace, is_end):
        if current_value > 21:
            if has_ace:
                current_value -= 10
            else:
                return current_value, has_ace, True
                    
        if current_value >= 17:
            return current_value, has_ace, is_end
        else:
            card = giveCard()
            if card == 1:
                if current_value <= 10:
                    return current_value+11, True, False
                return current_value+1, True, False
            else:
                return current_value+card, has_ace, False
        
    def play(self, rounds=1000):
        # hit 2 cards each
        dealer_value, player_value = 0, 0
        show_card = 0
        
        for _ in range(2):
            